# Sprint 3: Pothole Segmentation and Metrology

**Project:** Road Defect Detection System (PROSIT 1)  
**Team Members:**
- Naa Lamle Boye
- Thomas Kojo Quarshie
- Chelsea Owusu
- Elijah Boateng

**Date:** 2024

## Purpose

This notebook implements pothole detection and measurement in top-down road images.

**What this notebook achieves:**
- Detects potholes in top-down road images using image segmentation
- Calculates pothole area in cm² using the known scale factor
- Computes additional metrics (perimeter, equivalent diameter, aspect ratio)
- Validates detected regions to filter false positives
- Generates reports with pothole measurements

**Prerequisites:** 
- Input images should be top-down views (from homography transformation)
- Scale factor must be known (default: 10 pixels = 1 cm)

**Output:** 
- Annotated images with detected potholes
- Pothole measurements (area, center, perimeter, etc.)
- Reports in dictionary or CSV format

## Step 1: Import Required Libraries

In [1]:
import numpy as np
import cv2
from scipy import ndimage

## Step 2: Pothole Detection

Detect potholes in top-down road images using adaptive thresholding and morphological operations.

In [2]:
def detect_potholes(image, scale_factor=10.0, min_area_cm2=10, max_area_cm2=10000):
    """
    Detect potholes in top-down road image.
    
    Potholes appear as dark regions in the image. This function:
    1. Converts image to grayscale
    2. Applies adaptive thresholding to handle varying lighting
    3. Uses morphological operations to clean up noise
    4. Finds contours and filters by area
    5. Validates that regions are darker than surroundings
    
    Args:
        image: Top-down road image (BGR format)
        scale_factor: Pixels per cm (default: 10 pixels = 1 cm)
        min_area_cm2: Minimum pothole area in cm² (filters small noise)
        max_area_cm2: Maximum pothole area in cm² (filters large regions)
    
    Returns:
        list: List of dictionaries with keys: id, area_cm2, center, center_cm, contour, bbox
    """
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Adaptive thresholding to handle varying lighting conditions
    # THRESH_BINARY_INV: dark regions (potholes) become white
    binary = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )
    
    # Morphological operations to clean up the binary image
    # CLOSE: fills small holes inside potholes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    # OPEN: removes small noise
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    
    # Additional opening with smaller kernel to remove tiny noise
    kernel_small = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel_small)
    
    # Find contours (boundaries of detected regions)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter and process contours
    potholes = []
    min_area_pixels = min_area_cm2 * (scale_factor ** 2)
    max_area_pixels = max_area_cm2 * (scale_factor ** 2)
    
    for i, contour in enumerate(contours):
        area_pixels = cv2.contourArea(contour)
        
        # Filter by area
        if min_area_pixels <= area_pixels <= max_area_pixels:
            # Calculate area in cm²
            area_cm2 = area_pixels / (scale_factor ** 2)
            
            # Calculate center coordinates using image moments
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
            else:
                cx, cy = 0, 0
            
            # Bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Additional validation: check if it looks like a pothole
            # (darker than surroundings, has depth-like appearance)
            if is_valid_pothole(image, contour, cx, cy):
                potholes.append({
                    'id': len(potholes) + 1,
                    'area_cm2': area_cm2,
                    'center': (cx, cy),
                    'center_cm': (cx / scale_factor, cy / scale_factor),
                    'contour': contour,
                    'bbox': (x, y, w, h)
                })
    
    return potholes

## Step 3: Pothole Validation

Validate detected regions to filter false positives by checking if they are darker than their surroundings.

In [3]:
def is_valid_pothole(image, contour, cx, cy, threshold_ratio=0.7):
    """
    Validate if detected region is likely a pothole.
    
    Potholes should be darker than their surroundings. This function:
    1. Calculates mean intensity inside the contour
    2. Calculates mean intensity in surrounding region
    3. Checks if inside is significantly darker than outside
    
    Args:
        image: Original image
        contour: Contour of potential pothole
        cx, cy: Center coordinates
        threshold_ratio: Ratio threshold for validation (default: 0.7)
                        Inside should be < 70% of outside intensity
    
    Returns:
        bool: True if likely a pothole
    """
    # Create mask for the contour region
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    
    # Get mean intensity inside the contour
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    inside_mean = cv2.mean(gray, mask=mask)[0]
    
    # Get surrounding region (expanded bounding box)
    x, y, w, h = cv2.boundingRect(contour)
    expanded_mask = np.zeros_like(mask)
    cv2.rectangle(expanded_mask, (max(0, x-w), max(0, y-h)), 
                  (min(image.shape[1], x+2*w), min(image.shape[0], y+2*h)), 255, -1)
    outside_mask = expanded_mask - mask
    outside_mean = cv2.mean(gray, mask=outside_mask)[0]
    
    # Pothole should be darker than surroundings
    if outside_mean > 0:
        return inside_mean < outside_mean * threshold_ratio
    
    return True

In [4]:
def calculate_pothole_metrics(potholes, scale_factor=10.0):
    """
    Calculate additional metrics for detected potholes.
    
    Args:
        potholes: List of pothole dictionaries from detect_potholes()
        scale_factor: Pixels per cm
    
    Returns:
        list: Enhanced pothole dictionaries with additional metrics:
              - perimeter_cm: Perimeter in centimeters
              - equivalent_diameter_cm: Diameter of circle with same area
              - aspect_ratio: Width/height ratio of bounding box
              - extent: Ratio of contour area to bounding box area
    """
    for pothole in potholes:
        contour = pothole['contour']
        
        # Perimeter in cm
        perimeter_pixels = cv2.arcLength(contour, True)
        pothole['perimeter_cm'] = perimeter_pixels / scale_factor
        
        # Equivalent diameter (diameter of circle with same area)
        area_cm2 = pothole['area_cm2']
        pothole['equivalent_diameter_cm'] = 2 * np.sqrt(area_cm2 / np.pi)
        
        # Aspect ratio (width/height of bounding box)
        x, y, w, h = pothole['bbox']
        pothole['aspect_ratio'] = float(w) / h if h > 0 else 1.0
        
        # Extent (ratio of contour area to bounding box area)
        # Higher extent means the contour fills more of its bounding box
        bbox_area = w * h
        contour_area = cv2.contourArea(contour)
        pothole['extent'] = contour_area / bbox_area if bbox_area > 0 else 0
    
    return potholes

## Step 5: Generate Reports

Generate reports for detected potholes in dictionary or CSV format.

In [5]:
def generate_report(potholes, frame_id, output_format='dict'):
    """
    Generate report for detected potholes.
    
    Args:
        potholes: List of pothole dictionaries
        frame_id: Frame identifier (e.g., frame number, timestamp)
        output_format: 'dict' or 'csv_row'
    
    Returns:
        dict or str: Report data
        - If 'dict': Returns dictionary with frame_id, num_potholes, and potholes list
        - If 'csv_row': Returns CSV-formatted string with one row per pothole
    """
    if output_format == 'dict':
        return {
            'frame_id': frame_id,
            'num_potholes': len(potholes),
            'potholes': potholes
        }
    elif output_format == 'csv_row':
        rows = []
        for pothole in potholes:
            cx, cy = pothole['center_cm']
            rows.append(f"{frame_id},{pothole['id']},{pothole['area_cm2']:.2f},{cx:.2f},{cy:.2f}")
        return '\n'.join(rows)

## Step 6: Test Segmentation Functions

Test the segmentation functions with a sample image and visualize results.

In [ ]:
# Create synthetic test image with dark regions (simulating potholes)
test_image = np.ones((400, 600, 3), dtype=np.uint8) * 150  # Light gray background

# Add dark regions (simulating potholes)
cv2.rectangle(test_image, (100, 100), (200, 200), (50, 50, 50), -1)
cv2.circle(test_image, (400, 300), 50, (60, 60, 60), -1)
cv2.ellipse(test_image, (500, 150), (40, 60), 30, 0, 360, (55, 55, 55), -1)

# Detect potholes
potholes = detect_potholes(test_image, scale_factor=10.0, min_area_cm2=5, max_area_cm2=10000)

print(f"Found {len(potholes)} pothole(s)")

if len(potholes) > 0:
    # Calculate additional metrics
    potholes = calculate_pothole_metrics(potholes, scale_factor=10.0)
    
    # Display pothole details
    print("\nPothole Details:")
    for pothole in potholes:
        print(f"  Pothole {pothole['id']}:")
        print(f"    Area: {pothole['area_cm2']:.2f} cm²")
        print(f"    Center: ({pothole['center_cm'][0]:.2f}, {pothole['center_cm'][1]:.2f}) cm")
        print(f"    Perimeter: {pothole['perimeter_cm']:.2f} cm")
        print(f"    Equivalent Diameter: {pothole['equivalent_diameter_cm']:.2f} cm")
    
    # Annotate image with detected potholes
    annotated = test_image.copy()
    for pothole in potholes:
        # Draw contour
        cv2.drawContours(annotated, [pothole['contour']], -1, (0, 255, 0), 2)
        # Draw center point
        cx, cy = pothole['center']
        cv2.circle(annotated, (cx, cy), 5, (0, 0, 255), -1)
        # Add label with ID and area
        cv2.putText(annotated, f"ID:{pothole['id']} {pothole['area_cm2']:.1f}cm²",
                   (cx+10, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Save annotated image
    output_path = "test_segmentation_output.png"
    cv2.imwrite(output_path, annotated)
    print(f"\nSaved annotated image to: {output_path}")
    
    # Test report generation
    report = generate_report(potholes, frame_id=0, output_format='dict')
    print(f"\nReport generated: {report['num_potholes']} potholes in frame {report['frame_id']}")
else:
    print("No potholes detected in test image")

Found 1 pothole(s)

Pothole Details:
  Pothole 1:
    Area: 99.92 cm²
    Center: (15.00, 15.00) cm
    Perimeter: 39.53 cm
    Equivalent Diameter: 11.28 cm

Saved annotated image to: test_segmentation_output.png
CSV report saved to: pothole_detection_report.csv

Report generated: 1 potholes in frame 0
